In [14]:
import pandas as pd
import numpy as np
DATA_DIR = "/Users/patrickkennedy/Desktop/Data_Science_MISC/Kaggle"
train = pd.read_csv(DATA_DIR + "/BNP_Paribas/train.csv")
test = pd.read_csv(DATA_DIR + "/BNP_Paribas/test.csv")
categorical = []

for col in train.columns:
    s = str(train[col][0])
    d = str(train[col][1])
    f = str(train[col][2])
    try:
        float(s)
        float(d)
        float(f)
    except ValueError:
        categorical.append(col)
for cat in categorical:
    train[cat] = pd.factorize(train[cat])[0]


train.fillna(-1, inplace=True)


from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop(["ID","target"],axis=1), train["target"], test_size=0.33, random_state=42)


#i lost all my stuff making the appropriate weighting for ensembles... i still get best score so far with single xgboost (not wrapper)

In [3]:
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

model = XGBClassifier(max_depth = 9, 
                      learning_rate = 0.01,
                      n_estimators = 250,
                      min_child_weight = 50,
                      subsample = 0.7,
                      colsample_bytree = 0.9,
                      objective = 'binary:logistic')

model.fit(X_train, y_train)
pred = model.predict_proba(X_test)[:,1]

print log_loss(y_test, pred)

0.473695519563


In [4]:
model = XGBClassifier(max_depth = 6, 
                      learning_rate = 0.1,
                      n_estimators = 250,
                      min_child_weight = 5,
                      subsample = 0.6,
                      colsample_bytree = 0.7,
                      objective = 'reg:logistic')

model.fit(X_train, y_train)
pred2 = model.predict_proba(X_test)[:,1]

print log_loss(y_test, pred2)

0.46935861595


In [19]:
from sknn.mlp import Classifier, Layer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

#pipeline = Pipeline([
#        ('min/max scaler', MinMaxScaler(feature_range=(-1.0, 1.0))),
#        ('neural network', Classifier(layers=[Layer("Softmax")], n_iter=25))])
#pipeline.fit(X_train.as_matrix(), y_train.values)

#pred3 = pipeline.predict_proba(X_test.as_matrix())[:,1]


nn = Classifier(
    layers=[
        Layer("Sigmoid", units=100),
        Layer("Softmax")],
    learning_rate=0.001,
    n_iter=50)
nn.fit(X_train, y_train)


print log_loss(y_test, pred3)

ImportError: ('The following error happened while compiling the node', Dot22(X, W), '\n', 'dlopen(/Users/patrickkennedy/.theano/compiledir_Darwin-15.3.0-x86_64-i386-64bit-i386-2.7.11-64/tmpGURPga/eb163660e6e45b373cd7909e14efd44a.so, 2): Library not loaded: libmkl_intel_lp64.dylib\n  Referenced from: /Users/patrickkennedy/.theano/compiledir_Darwin-15.3.0-x86_64-i386-64bit-i386-2.7.11-64/tmpGURPga/eb163660e6e45b373cd7909e14efd44a.so\n  Reason: image not found', '[Dot22(X, W)]')

In [9]:
X_train.as_matrix()

array([[ -1.        ,  -1.        ,   0.        , ...,   0.        ,
         -1.        ,  -1.        ],
       [  1.87086644,  12.2929544 ,   0.        , ...,   0.        ,
          1.87142846,   1.98473273],
       [ -1.        ,  -1.        ,   0.        , ...,   0.        ,
         -1.        ,  -1.        ],
       ..., 
       [ -1.        ,  -1.        ,   0.        , ...,   3.        ,
         -1.        ,  -1.        ],
       [  1.24202723,  11.61352056,   0.        , ...,   0.        ,
          1.42776295,   1.58730066],
       [  0.6747719 ,  11.69122718,   0.        , ...,   0.        ,
          0.70000024,   0.9523803 ]])

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import ensemble
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
import random


#print('Load data...')
DATA_DIR = "/Users/patrickkennedy/Desktop/Data_Science_MISC/Kaggle"
train = pd.read_csv(DATA_DIR + "/BNP_Paribas/train.csv")
test = pd.read_csv(DATA_DIR + "/BNP_Paribas/test.csv")

target = train['target'].values

train = train.drop(['ID','target'],axis=1)
id_test = test['ID'].values
test = test.drop(['ID'],axis=1)

y_pred = []
holder = []

#X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

for i in range(500):
    
    dummy = random.randint(1,10000)
    x = True
    while x == True:
        print dummy, str(len(holder)+1), holder
        #print holder
        if dummy in holder:
            dummy = random.randint(1,10000)
        else:
            x = False
    holder.append(dummy)
    
    random.seed(dummy)
    
    
    #print('Clearing...')
    for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
        if train_series.dtype == 'O':
            #for objects: factorize
            train[train_name], tmp_indexer = pd.factorize(train[train_name])
            test[test_name] = tmp_indexer.get_indexer(test[test_name])
            #but now we have -1 values (NaN)
        else:
            #for int or float: fill NaN
            tmp_len = len(train[train_series.isnull()])
            if tmp_len>0:
                #print "mean", train_series.mean()
                train.loc[train_series.isnull(), train_name] = -9999 #train_series.mean()
            #and Test
            tmp_len = len(test[test_series.isnull()])
            if tmp_len>0:
                test.loc[test_series.isnull(), test_name] = -9999 #train_series.mean()  #TODO

    X_train = train
    X_test = test

    #extc = ExtraTreesClassifier(n_estimators=700,max_features= 50,criterion= 'entropy',min_samples_split= 5,
    #                            max_depth= 10, min_samples_leaf= 5)      
    
    #extc = ExtraTreesClassifier(n_estimators=random.randint(200,1000), 
    #                            max_features=random.randint(5,100),
    #                            criterion='entropy',
    #                            max_depth=random.randint(2,20))
    
    #print('Fit....')
    #adaextc = AdaBoostClassifier(base_estimator = ExtraTreesClassifier())
    #adaextc.fit(X_train, target) 
    
    extc = ExtraTreesClassifier()
    extc.fit(X_train, target)
    
    if i%10==0:
        logm = LogisticRegression()
        logm.fit(X_train, target)
        y_pred.append(logm.predict_proba(X_test)[:,1])
        
        adalogm = AdaBoostClassifier(LogisticRegression())
        adalogm.fit(X_train, target)
        y_pred.append(adalogm.predict_proba(X_test)[:,1])
    
    if i%5==0:
        gbc = GradientBoostingClassifier()
        gbc.fit(X_train, target)
        y_pred.append(gbc.predict_proba(X_test)[:,1])
        
        adagbc = AdaBoostClassifier(GradientBoostingClassifier())
        adagbc.fit(X_train, target)
        y_pred.append(adagbc.predict_proba(X_test)[:,1])
    

    #print('Predict...')
    y_pred.append(extc.predict_proba(X_test)[:,1])
    #print y_pred

    #print log_loss(y_test, y_pred[i])

pd.DataFrame({"ID": id_test, "PredictedProb": np.mean(y_pred, axis=0)}).to_csv('extra_trees_and_log_and_gradientboost_with adas_jitteredrandomstate_500iterations.csv',index=False)